In [1]:
import pandas as pd

# Extracting and transforming room_type csv

In [4]:
# Creating path for room_type file 

file_path = 'Resources/airbnb_room_type.csv'

In [ ]:
#loading file into data frame

room_type_df = pd.read_csv(file_path)
#displaying dataframe
room_type_df.head()

,listing_id,description,room_type
0,2595,Skylit Midtown Castle,Entire home/apt
1,3831,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt


In [6]:
#removing extra characters from room_type
room_type_df['room_type'] = room_type_df['room_type'].str.strip(")")
#Verifying changes
print(room_type_df['room_type'].unique())

['Entire home/apt' 'private room' 'Private room' 'entire home/apt'
 'PRIVATE ROOM' 'shared room' 'ENTIRE HOME/APT' 'Shared room'
 'SHARED ROOM']


In [ ]:
# Putting all values in the room_type column in lower case
room_type_df['room_type'] = room_type_df['room_type'].str.lower()
# confirming changes
print(room_type_df['room_type'].unique())

['entire home/apt' 'private room' 'shared room']


In [12]:
#removing na values from room_type column
room_type_df = room_type_df.dropna(subset=['room_type'])


In [15]:
#removing duplicate rows 
room_type_df = room_type_df.drop_duplicates()
room_type_df.head()

,listing_id,description,room_type
0,2595,Skylit Midtown Castle,entire home/apt
1,3831,Cozy Entire Floor of Brownstone,entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,entire home/apt
